## Modèle avancé

In [4]:
from helpers import *

In [5]:
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")

In [6]:
cols = ['sentiment', 'timestamp', 'date', "query", "username", "comment"]  
df = pd.read_csv("./../../input/training.1600000.processed.noemoticon.csv", header=None, names=cols)

df = df.sample(n=100000, random_state=42)
len_df = len(df)
print(df.shape)
df.head()

(100000, 6)


,sentiment,timestamp,date,query,username,comment
541200,0,2200003196,Tue Jun 16 18:18:12 PDT 2009,NO_QUERY,LaLaLindsey0609,@chrishasboobs AHHH I HOPE YOUR OK!!!
750,0,1467998485,Mon Apr 06 23:11:14 PDT 2009,NO_QUERY,sexygrneyes,"@misstoriblack cool , i have no tweet apps fo..."
766711,0,2300048954,Tue Jun 23 13:40:11 PDT 2009,NO_QUERY,sammydearr,@TiannaChaos i know just family drama. its la...
285055,0,1993474027,Mon Jun 01 10:26:07 PDT 2009,NO_QUERY,Lamb_Leanne,School email won't open and I have geography ...
705995,0,2256550904,Sat Jun 20 12:56:51 PDT 2009,NO_QUERY,yogicerdito,upper airways problem


In [7]:
df = clean_df(df)
print(df.shape)
df.head()

Processing: 100%|██████████| 100000/100000 [00:00<00:00, 784153.54it/s]


(92469, 5)


,comment,sentiment,comment_clean,words_nb,words_nb_clean
541200,@chrishasboobs AHHH I HOPE YOUR OK!!!,0,ahhh i hope your ok,6,5
750,"@misstoriblack cool , i have no tweet apps fo...",0,cool i have no tweet apps for my razr,12,9
766711,@TiannaChaos i know just family drama. its la...,0,i know just family drama its lamehey next time...,27,26
285055,School email won't open and I have geography ...,0,school email wont open and i have geography st...,16,15
379611,Going to miss Pastor's sermon on Faith...,0,going to miss pastors sermon on faith,7,7


### Word2Vec

In [8]:
df_w2vec = df.copy()

config = {
    "vector_size": 100,
    # Set input length to be the max from the number of words
    "input_length": df_w2vec["words_nb_clean"].max(),
    # Word before & after for word2vec
    "window": 5,
    # Max number of workers
    "workers": 8,
    # Vocal length
    "vocab_length": 14000
}

(model_w2vec, tokenizer_w2vec, callbacks_w2vec, X_w2vec, y_w2vec) = get_w2vec_data(config, df_w2vec)


W2vec Vocabulary Length (you can adjust the vocal length): 8196
Embedding Matrix Shape: (14000, 100)
CONFIG {'vector_size': 100, 'input_length': 40, 'window': 5, 'workers': 8, 'vocab_length': 14000}


/Users/m/dev/6be709c0/oc-ai-engineer/OC/P7/output/tf-nightly/venv/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:81: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
2024-02-07 10:43:49.420407: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-02-07 10:43:49.420468: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 24.00 GB
2024-02-07 10:43:49.420481: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 8.00 GB
2024-02-07 10:43:49.420541: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-02-07 10:43:49.420580: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created

In [9]:
model_w2vec.summary()

Model: "Sentiment_Model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape              ┃    Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ layer_embedding (Embedding)     │ (None, 40, 100)           │  1,400,000 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ layer_ltsm (LSTM)               │ (None, 40, 100)           │     80,400 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ layer_conv1d (Conv1D)           │ (None, 36, 100)           │     50,100 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ later_globalmaxpool1d           │ (None, 100)               │          0 │
│ (GlobalMaxPooling1D)            │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ layer_dense_1 (Dense)           │ (None, 16)                │      1,616 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ layer_dense_2 (Dense)           │ (None, 1)                 │         17 │
└─────────────────────────────────┴───────────────────────────┴────────────┘

 Total params: 1,532,133 (5.84 MB)

 Trainable params: 1,532,133 (5.84 MB)

 Non-trainable params: 0 (0.00 B)

In [10]:
space = {
    "input_length": config["input_length"],
    'experiment_name': f"NIGHTLY - Tweet Sentiment - {len_df} - w2vec",
    'batch_size': hp.choice('batch_size', [128, 256]),  
    # 'epochs': hp.choice('epochs', [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]),  
    'epochs': hp.choice('epochs', [1]),  
    'learning_rate': hp.uniform('learning_rate', 0.0001, 0.001),  
}

In [11]:
trials = Trials()
fmin(  
    fn=lambda params: objective(params, X_w2vec, y_w2vec, model_w2vec, callbacks_w2vec, tokenizer_w2vec),  
    space=space,  
    algo=tpe.suggest,  
    max_evals=1,
    trials=trials  
)

Running one fit with the params:                     
{'batch_size': 256, 'epochs': 1, 'experiment_name': 'NIGHTLY - Tweet Sentiment - 100000 - w2vec', 'input_length': 40, 'learning_rate': 0.0005738060634476003}
  0%|          | 0/1 [00:00<?, ?trial/s, best loss=?]

2024-02-07 10:43:50.151089: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-02-07 10:43:50.437019: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


  1/228 ━━━━━━━━━━━━━━━━━━━━ 14:25 4s/step - accuracy: 0.5039 - loss: 0.6983
  2/228 ━━━━━━━━━━━━━━━━━━━━ 5:44 2s/step - accuracy: 0.5117 - loss: 0.6955 
  3/228 ━━━━━━━━━━━━━━━━━━━━ 5:42 2s/step - accuracy: 0.5143 - loss: 0.6946
  4/228 ━━━━━━━━━━━━━━━━━━━━ 5:42 2s/step - accuracy: 0.5176 - loss: 0.6939
  5/228 ━━━━━━━━━━━━━━━━━━━━ 5:40 2s/step - accuracy: 0.5216 - loss: 0.6927
  6/228 ━━━━━━━━━━━━━━━━━━━━ 5:39 2s/step - accuracy: 0.5244 - loss: 0.6917
  7/228 ━━━━━━━━━━━━━━━━━━━━ 5:37 2s/step - accuracy: 0.5262 - loss: 0.6911
  8/228 ━━━━━━━━━━━━━━━━━━━━ 5:35 2s/step - accuracy: 0.5281 - loss: 0.6904
  9/228 ━━━━━━━━━━━━━━━━━━━━ 5:38 2s/step - accuracy: 0.5301 - loss: 0.6898
 10/228 ━━━━━━━━━━━━━━━━━━━━ 5:37 2s/step - accuracy: 0.5323 - loss: 0.6891
 11/228 ━━━━━━━━━━━━━━━━━━━━ 5:35 2s/step - accuracy: 0.5349 - loss: 0.6883
 12/228 ━━━━━━━━━━━━━━━━━━━━ 5:33 2s/step - accuracy: 0.5375 - loss: 0.6876
 13/228 ━━━━━━━━━━━━━━━━━━━━ 5:32 2s/step - accuracy: 0.5400 - loss: 0.6869
 14/228 ━━

2024-02-07 10:50:04.157730: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


228/228 ━━━━━━━━━━━━━━━━━━━━ 378s 2s/step - accuracy: 0.6505 - loss: 0.6210 - val_accuracy: 0.7429 - val_loss: 0.5200 - learning_rate: 5.7381e-04

   1/2023 ━━━━━━━━━━━━━━━━━━━━ 8:07 241ms/step
  0%|          | 0/1 [06:17<?, ?trial/s, best loss=?]

2024-02-07 10:50:07.418639: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


   2/2023 ━━━━━━━━━━━━━━━━━━━━ 2:38 79ms/step 
   3/2023 ━━━━━━━━━━━━━━━━━━━━ 2:46 82ms/step
   4/2023 ━━━━━━━━━━━━━━━━━━━━ 2:42 81ms/step
   5/2023 ━━━━━━━━━━━━━━━━━━━━ 2:42 81ms/step
   6/2023 ━━━━━━━━━━━━━━━━━━━━ 2:40 79ms/step
   7/2023 ━━━━━━━━━━━━━━━━━━━━ 2:40 80ms/step
   8/2023 ━━━━━━━━━━━━━━━━━━━━ 2:40 80ms/step
   9/2023 ━━━━━━━━━━━━━━━━━━━━ 2:40 80ms/step
  10/2023 ━━━━━━━━━━━━━━━━━━━━ 2:45 82ms/step
  11/2023 ━━━━━━━━━━━━━━━━━━━━ 2:47 83ms/step
  12/2023 ━━━━━━━━━━━━━━━━━━━━ 2:45 83ms/step
  13/2023 ━━━━━━━━━━━━━━━━━━━━ 2:44 82ms/step
  14/2023 ━━━━━━━━━━━━━━━━━━━━ 2:44 82ms/step
  15/2023 ━━━━━━━━━━━━━━━━━━━━ 2:43 81ms/step
  16/2023 ━━━━━━━━━━━━━━━━━━━━ 2:43 81ms/step
  17/2023 ━━━━━━━━━━━━━━━━━━━━ 2:42 81ms/step
  18/2023 ━━━━━━━━━━━━━━━━━━━━ 2:42 81ms/step
  19/2023 ━━━━━━━━━━━━━━━━━━━━ 2:42 81ms/step
  20/2023 ━━━━━━━━━━━━━━━━━━━━ 2:42 81ms/step
  21/2023 ━━━━━━━━━━━━━━━━━━━━ 2:42 81ms/step
  22/2023 ━━━━━━━━━━━━━━━━━━━━ 2:41 81ms/step
  23/2023 ━━━━━━━━━━━━━━━━━━━━ 2:

2024-02-07 10:53:31.664055: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


2023/2023 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
2023/2023 ━━━━━━━━━━━━━━━━━━━━ 205s 101ms/step

Log done                                             
100%|██████████| 1/1 [09:59<00:00, 599.26s/trial, best loss: -0.7429322004318237]


{'batch_size': 1, 'epochs': 0, 'learning_rate': 0.0005738060634476003}

In [12]:
from keras.models import load_model  

params = json.load(open("./params.json", 'r', encoding='utf-8'))
model = load_model("./model.keras")
def pad_sequences(sequence, maxlen, value=0):  
    return np.array([np.pad(s[:maxlen], (max(0, maxlen-len(s)), 0), 'constant', constant_values=value) if len(s) < maxlen else s[:maxlen] for s in sequence])  
  
def predict(comment, model, vector_model):
    # We need to pad sequences to ensure uniform input size  
    sequence = tokenizer_w2vec.texts_to_sequences([comment])
    padded_sequence = pad_sequences(sequence, maxlen=int(params["input_length"]))

    prediction = model.predict(padded_sequence)
    # print(prediction)
    sentiment = prediction.astype(float)[0][0]
    return sentiment.item()
    
def comment_to_vec(comment, model):
    vec = np.zeros(100)
    num_words = 0
    for word in comment:
        if word in model.wv:
            vec += model.wv[word]
            num_words += 1
    if num_words > 0:
        vec /= num_words
    return vec

print(predict("I am so sad, this is very bad news, terrible!", model, tokenizer_w2vec))
print(predict("I am so happy, this is very good news, congrats!", model, tokenizer_w2vec))


2024-02-07 10:53:49.108031: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 267ms/step
0.13652688264846802
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
0.7164327502250671


In [20]:
# We need to pad sequences to ensure uniform input size  
sequence = tokenizer_w2vec.texts_to_sequences(df["comment_clean"])
padded_sequence = pad_sequences(sequence, maxlen=int(params["input_length"]))
prediction = model.predict(padded_sequence)


2024-02-07 10:57:34.629413: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


2890/2890 ━━━━━━━━━━━━━━━━━━━━ 426s 147ms/step


In [21]:
y_pred_rounded = (prediction > 0.5).astype(int)  # Convert probabilities to binary predictions (0 or 1)
accuracy_score(df["sentiment"], y_pred_rounded)

0.7426272588651331

### Glove

In [ ]:
df_glove = df.copy()

config = {
    # Best to use 100 since I'm using glove 100 of pretrained data
    "vector_size": 100,
    # Set input length to be the max from the number of words
    "input_length": df_glove["words_nb_clean"].max(),
    # Word before & after for word2vec
    "window": 5,
    # Max number of workers
    "workers": 8,
    # Vocal length
    "vocab_length": 14000
}

(model_glove, tokenizer_glove, callbacks_glove, X_glove, y_glove) = get_glove_data(config, df_glove)


In [ ]:
model_glove.summary()

In [ ]:
space = {
    "input_length": config["input_length"],
    'experiment_name': f"Tweet Sentiment - {len_df} - glove",
    'batch_size': hp.choice('batch_size', [128, 256]),  
    'epochs': hp.choice('epochs', [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]),  
    'learning_rate': hp.uniform('learning_rate', 0.0001, 0.001),  
}

In [ ]:
trials = Trials()
fmin(  
    fn=lambda params: objective(params, X_glove, y_glove, model_glove, callbacks_glove, tokenizer_glove),  
    space=space,  
    algo=tpe.suggest,
    max_evals=4,
    trials=trials  
)